In [ ]:
import numpy as np
import pandas as pd
from IPython.display import clear_output
from tqdm import tqdm
import urllib.request


In [ ]:
train_docs = pd.read_csv("../Data/nfcorpus/raw/doc_dump.txt", sep='\t', index_col=False, header=None )
train_docs = train_docs.rename(columns={ 0: "index", 1: "URL"})

train_docs = train_docs.set_index('index')
train_docs['Topics'] = None
#train_docs.docs = np.expand_dims(np.squeeze([doc.split(" ") for doc in train_docs.docs.values]), axis=1)


In [ ]:
not_processed = []
errors = []
for index, row in tqdm(train_docs.iterrows(), total=len(train_docs)):
    try:
        with urllib.request.urlopen(row.URL) as f:
            topics = []
            isTopic = False
            expect_topic_value = False
            html = f.read().decode('utf-8')
            for html_line in html.splitlines():
                if '<div id="mesh-terms" class="mesh-terms keywords-section">' in html_line:
                    isTopic = True
                if isTopic:
                    if '</li></ul>' in html_line:
                        isTopic = False
                        continue
                    if html_line == '' or html_line.isspace():
                        continue

                    if expect_topic_value:
                        if '</li><li>' not in html_line:
                            topics.append(html_line.strip().lower().replace('*', ''))
                    if '<ul class="keywords-list"><li>' in html_line:
                        expect_topic_value = True

        train_docs['Topics'].loc[index] = topics
    except Exception as e:
        errors.append(e)
        not_processed.append(index)
        train_docs['Topics'].loc[index] = []


train_docs = train_docs.drop(columns=['URL', 2, 3])
train_docs.to_csv('../Data/clean/med_documents.csv')

100%|████████████████████████████████████████████████████████████████████████████| 5371/5371 [1:14:45<00:00,  1.20it/s]


In [ ]:
train_docs = train_docs.drop(columns=['URL', 2, 3])

train_docs.head(60)


,Topics
index,
MED-1,"[acrylamide / blood, adult, birth weight, chro..."
MED-2,"[acrylamide / chemistry, acrylamide / toxicity..."
MED-3,"[acrylamide / administration &amp; dosage, acr..."
MED-4,"[adult, aged, breast neoplasms / etiology, bre..."
MED-5,"[adult, aged, aged, 80 and over, body mass ind..."
MED-6,"[adult, aged, case-control studies, cooking, d..."
MED-7,"[amines / adverse effects, amines / chemistry,..."
MED-8,"[acrylamide / adverse effects, acrylamide / an..."
MED-9,"[apoptosis / drug effects, apoptosis / physiol..."


In [ ]:
train_docs['Topics'] = train_docs['Topics'].apply(lambda x: x.split('+')  if x is not None else x)


In [ ]:
train_docs['Topics'][['MED-50']].values[0]

['arsenic / analysis',
 'child',
 'child, preschool',
 'edible grain / chemistry',
 'food contamination / analysis',
 'humans',
 'infant',
 'oryza / chemistry',
 'sweetening agents']

In [ ]:
train_docs['Topics'][['MED-1']].values[0]

['acrylamide / blood',
 'adult',
 'birth weight',
 'chromatography, liquid',
 'cohort studies',
 'diet',
 'environmental exposure',
 'environmental monitoring',
 'environmental pollutants / blood',
 'epoxy compounds / blood',
 'europe / epidemiology',
 'female',
 'fetal blood / chemistry',
 'head / anatomy &amp; histology',
 'hemoglobins / metabolism',
 'humans',
 'infant, low birth weight',
 'infant, newborn',
 'mass spectrometry',
 'maternal exposure',
 'pregnancy',
 'prenatal exposure delayed effects / chemically induced',
 'prenatal exposure delayed effects / epidemiology',
 'prospective studies',
 'regression analysis',
 'surveys and questionnaires']

In [ ]:
train_docs

,URL,2,3,Topics
index,,,,
MED-1,http://www.ncbi.nlm.nih.gov/pubmed/23092936,"Birth Weight, Head Circumference, and Prenatal...",Abstract Background: Acrylamide is a common di...,"[acrylamide / blood, adult, birth weight, chro..."
MED-2,http://www.ncbi.nlm.nih.gov/pubmed/22809476,A statistical regression model for the estimat...,Abstract Human exposure to acrylamide (AA) thr...,"[acrylamide / chemistry, acrylamide / toxicity..."
MED-3,http://www.ncbi.nlm.nih.gov/pubmed/19158207,Chronic intake of potato chips in humans incre...,Abstract BACKGROUND: Relatively high concentra...,"[acrylamide / administration &amp; dosage, acr..."
MED-4,http://www.ncbi.nlm.nih.gov/pubmed/23651876,Dietary patterns and breast cancer risk among ...,Abstract OBJECTIVE: Breast cancer is the most ...,"[adult, aged, breast neoplasms / etiology, bre..."
MED-5,http://www.ncbi.nlm.nih.gov/pubmed/16332665,Empirically derived dietary patterns and risk ...,Abstract BACKGROUND: Inconsistent associations...,"[adult, aged, aged, 80 and over, body mass ind..."
...,...,...,...,...
MED-5367,http://www.ncbi.nlm.nih.gov/pubmed/21929378,The relationship between plasma carotenoids an...,Abstract Objective We examined the cross-secti...,None
MED-5368,http://www.ncbi.nlm.nih.gov/pubmed/24812159,Suicide Mortality in Relation to Dietary Intak...,Abstract Intake of n-3 and n-6 polyunsaturated...,None
MED-5369,http://www.ncbi.nlm.nih.gov/pubmed/12803408,Suicide mortality in the European Union. - Pub...,Abstract BACKGROUND: There are an estimated on...,None
